In [4]:
import pandas as pd

In [5]:
def clean_gazetteer(columns: list, featureclass: list, numLines: int=None, **kwargs):

    if numLines != None:
        chunksize = numLines // 1000
    else:
        chunksize = 1000

    iter_csv = pd.read_csv(
        **kwargs,
        iterator=True,
        chunksize=chunksize,
        low_memory=False
    )

    # df = pd.concat([chunk[chunk['featureclass'] in featureclass] for chunk in iter_csv])

    lyst = []

    for i, chunk in enumerate(iter_csv):

        # ATTENTION: Two conditions
        lyst.append(chunk[(chunk["FC"].isin(featureclass))][columns]) # & (chunk["NT"] == "C")])

        if i % 10 == 0:
            print(f"{i} of {int(numLines/chunksize)} chunks ({round((i/int(numLines/chunksize)), 3) * 100} %)", end="\r", flush=True)

    print()
    print("Now concatenating chunks...")
    df = pd.concat(lyst)


    return df

In [6]:
    # world = clean_gazetteer(
    #     path="data/geonames/allCountries.txt",
    #     featureclass=["A", "T", "P"],
    #     names=["geonameid", "name", "asciiname", "alternatenames", "latitude", "longitude",
    #             "featureclass", "featurecode", "countrycode", "cc2", "admin1code", "admin2code",
    #             "admin3code", "admin4code", "population", "elevation",
    #             "dem", "timezone", "modificationdate"]
    # )


world = clean_gazetteer(
    columns=['LAT', 'LONG', 'FC', 'CC1', 'NT', 'SORT_NAME_RO', 'FULL_NAME_RO',
    'FULL_NAME_ND_RO', 'SORT_NAME_RG', 'FULL_NAME_RG', 'FULL_NAME_ND_RG'],
    featureclass=["A", "P"],# , "P"],
    numLines=13516708,
    filepath_or_buffer="data/nga/countries.txt",
    delimiter="\t",
    header=0
)

    

1000 of 1000 chunks (100.0 %)99999999 %))
Now concatenating chunks...


In [7]:
world.to_csv("data/nga/countries_administrative_populated.csv")